Having just read the book, "The Man Who Solved The Markets: How Jim Simons Launched The Quant Revolution", I've become interested in the applications of pattern recognition in financial markets. While the book remains far more of a biography than an analysis of algorithmic trading strategies, it still mentions a few of the mathematical tools Simons employed to grow the Medallion Fund. One such tool is Markov models, which describe mathematical systems that hop from one "state" (a situation or set of values) to another. Markov models are complex systems and trying to implement one here remains far beyond my elementary coding abilities. That said, I have decided to try and get to grips with the basics of AI / pattern recognition by applying similar principles to the very simple game: Rock, Paper, Scissors.

Although rock-paper-scissors (RPS) may seem like a trivial game, it actually involves the hard computational problem of temporal pattern recognition. This problem is fundamental to the fields of machine learning, artificial intelligence, and data compression.

To address this problem myself, I've decided to implement the following pattern recognition strategy.

Assume the list below shows the last 20 hands played by an opponent in a game of RPS.

['S','R','R','P','S','P','R','P','S','P','S','P','S','P','R','P','S','P','S','R'] ?

To judge what is most likely to be played in the 21st hand, we can start by looking at the 20th, Rock, and see what the distribution of hands played following a Rock is. From there, we can look at the combination of the 19th and 20th hand, Scissors then Rock, and see what the distribution of hands played following a Scissors then a Rock. We can then train the code to repeat this process for all patterns in length up to half of the total length of the list of hands already played.

For each round of 'next hand guessing', we will be able to return a table showing, 1. The pattern preceding the next (unknown hand), whether that pattern is either the last hand played or a sequence of the last 20 hands, 2. The amount of times this pattern has come up in the game so far, excluding this instance, and 3. Most importantly, the percentage distribution of Rock / Paper / Scissors that has been played following instances in which this pattern has previously appeared.

The table for the set of 20 hands above would look as follows where %R, %P and %S are the hands that would follow the pattern;

In [82]:
# output table from sample sequence

                              Pattern % Rock % Paper % Scissors  N
1                                 [R]   25.0    75.0        0.0  4
2                              [S, R]  100.0     0.0        0.0  1
3                           [P, S, R]      -       -          -  0
4                        [S, P, S, R]      -       -          -  0
5                     [P, S, P, S, R]      -       -          -  0
6                  [R, P, S, P, S, R]      -       -          -  0
7               [P, R, P, S, P, S, R]      -       -          -  0
8            [S, P, R, P, S, P, S, R]      -       -          -  0
9         [P, S, P, R, P, S, P, S, R]      -       -          -  0
10     [S, P, S, P, R, P, S, P, S, R]      -       -          -  0
11  [P, S, P, S, P, R, P, S, P, S, R]      -       -          -  0


This strategy in of itself is by no means advanced, however there are certain approaches we can take to develop it further. First, we need a way to combine all the columns / patterns in deciding what hand to play next. To do that, we can convert the % figures back to integers. For each hand (R, P or S), we can then add up how many times each has come up in each pattern. Using a sum function, we can decide which hand to play. The output would look as follows. 

In [72]:
# output table with N values

                           Pattern  % Rock  % Paper  % Scissors    N  N(Rock)  \
1                                R    25.0     75.0         0.0  4.0      1.0   
2                              S-R   100.0      0.0         0.0  1.0      1.0   
3                        [P, S, R]     0.0      0.0         0.0  0.0      0.0   
4                     [S, P, S, R]     0.0      0.0         0.0  0.0      0.0   
5                  [P, S, P, S, R]     0.0      0.0         0.0  0.0      0.0   
6               [R, P, S, P, S, R]     0.0      0.0         0.0  0.0      0.0   
7            [P, R, P, S, P, S, R]     0.0      0.0         0.0  0.0      0.0   
8         [S, P, R, P, S, P, S, R]     0.0      0.0         0.0  0.0      0.0   
9      [P, S, P, R, P, S, P, S, R]     0.0      0.0         0.0  0.0      0.0   
10  [S, P, S, P, R, P, S, P, S, R]     0.0      0.0         0.0  0.0      0.0   

    N(Paper)  N(Scissors)  
1        3.0          0.0  
2        0.0          0.0  
3        0.0          0.

After doing this analysis, we can then sum each of the N(Rock) / N(Paper) / N(Scissors) rows and in doing so find the most likely hand that would follow the historic patterns. We can then select our own response that will (hopefully) beat this hand.

This is certainly a simple approach and there is much room for further improvemnt. Firstly, we could build in memory to the model so that recent historical patterns hold more weight than those idenitfied in the earlt stages of the game as an opponent may change his strategy. Second, we could devise a more advanced way of evaluating the table outputs, perhaps using significance tests for the data on each row / pattern.

In [1]:
import pandas as pd
import numpy as np
import random

Here is the algorithm

In [73]:
patterns_master = pd.DataFrame(columns = ['Pattern','% Rock','% Paper','% Scissors','N'])
def identifypattern(array,n):
    
    patterns = pd.DataFrame(columns = ['Pattern','% Rock','% Paper','% Scissors','N'])
    
    def countX(lst, x):
        count = 0
        for ele in lst:
            if (ele == x):
                count = count + 1
        return count

    for i in range(0, n):
        
        # select pattern
        pattern = array[len(array)-i-1:len(array)]
        
        # find all instances where first element of pattern occurs in array
        possibles = np.where(array == pattern[0])[0]
        
        # don't want one of the possibles to include the pattern itself
        possibles = [x for x in possibles if x < len(array)-len(pattern)]
        solns,hands = ([] for i in range(2))
        
        # now find all instances of the pattern
        for p in possibles:
            check = array[p:p+len(pattern)]
            if len(check) == len(pattern): 
                if np.all(check == pattern):
                    solns.append(p)
        
        if len(solns) == 0: # i.e. no instances of this pattern have been previously recorded
            row = [pattern,0,0,0,0] # better to use np.NaN then 0?
            patterns.loc[len(patterns)] = row
        
        else: 
            # retrieve next hand played for all previous instances of pattern
            for sol in solns:
                hands.append(array[sol+len(pattern)])
                
            # convert to percentages
            hands_count = [countX(hands,'R'),countX(hands,'P'),countX(hands,'S')] 
            hands_percent = [x / sum(hands_count) for x in hands_count]
            hands_percent = [ele * 100 for ele in hands_percent]
            hands_percent.append(round(len(solns)))
            hands_percent = np.array(hands_percent, dtype = "object")
            pattern = pattern.tolist()
            pattern = '-'.join(pattern)
            row = np.insert(hands_percent,0,pattern)
            row = row.tolist()
            patterns.loc[len(patterns)] = row  
    patterns.index = np.arange(1, len(patterns) + 1)
    
    # convert values to float
    patterns["% Rock"] = pd.to_numeric(patterns["% Rock"], downcast="float")
    patterns["% Paper"] = pd.to_numeric(patterns["% Paper"], downcast="float")
    patterns["% Scissors"] = pd.to_numeric(patterns["% Scissors"], downcast="float")
    patterns["N"] = pd.to_numeric(patterns["N"], downcast="float")
  
    # finding the EV of each hand
    patterns["N(Rock)"] = patterns["% Rock"] * patterns["N"] * 0.01
    patterns["N(Paper)"] = patterns["% Paper"] * patterns["N"] * 0.01
    patterns["N(Scissors)"] = patterns["% Scissors"] * patterns["N"] * 0.01
    
    print(patterns)
    
    outcome_dict = {'P': patterns['N(Rock)'].sum(), 'S': patterns['N(Paper)'].sum(), 'R': patterns['N(Scissors)'].sum()}
    global outcome, patterns_master
    outcome = max(outcome_dict, key=outcome_dict.get)
    # print(outcome)
    
    # add the analysis from this round into the master analysis dataframe
    patterns_master = patterns_master.append(patterns)
    patterns_master.loc[len(patterns_master)] = ['End of analysis',0,0,0,0,0,0,0]

# build memory into model

Below is a master function that controls how many patterns to analyse for each round of a game.

In [74]:
def markov1(array):
    if len(array) == 1:
        global outcome
        outcome = random.choice(['R','P','S'])
        print(outcome)
        
    else:
        if len(array) < 40:
            if len(array) % 2 == 0:
                n = int(len(array)/2)
            else: 
                n = int((len(array)/2)-0.5)
            print(n)
            identifypattern(array,n)
            
        else:
            identifypattern(array,20)          

Here is the functionality for a human opponent to play against the program.

In [ ]:
user_array = np.array([], dtype = "object")

rounds = input("How many rounds do you want to play: ")

computer_score = 0
opponent_score = 0

for _ in range(int(rounds)):
    user_action = input("Enter a choice (R, P, S): ")
    user_array = np.append(user_array, user_action)
    print(user_array)
    markov1(user_array)
    if user_action == 'R' and outcome == 'P': 
        computer_score += 1
    if user_action == 'S' and outcome == 'R':
        computer_score += 1
    if user_action == 'P' and outcome == 'S':
        computer_score += 1
    if user_action == 'R' and outcome == 'S':
        opponent_score += 1
    if user_action == 'S' and outcome == 'P':
        opponent_score += 1
    if user_action == 'P' and outcome == 'R':
        opponent_score += 1
    print("Computer action: {}".format(outcome))
    print("Computer score: {}, Your score: {}".format(computer_score, opponent_score))